# Measuring the total cold gas mass in Aurora's M87 observation
Grant Tremblay (grant.tremblay@yale.edu)

## tl,dr
I get $M_{\mathrm{H}_2} = (9.04 \pm 1.3) \times 10^5~M_\odot$ assuming a Milky Way $X_\mathrm{CO}$ and a CO(2-1) to CO(1-0) flux density ratio of 3.2

## The Details

Following (e.g.) Bollato+13, the total mass of H$_2$ scales with the CO(1-0) emission integral $S_\mathrm{CO}\Delta V$ as:

$M_\mathrm{mol}  = \frac{1.05 \times 10^4}{3.2} ~ \left(\frac{X_{\mathrm{CO}}}{2\times 10^{20}~\frac{\mathrm{cm}^{-2}}{\mathrm{K~km~s}^{-1}}}\right) \left( \frac{1}{1+z}\right) \left(\frac{S_{\mathrm{CO}}\Delta v}{\mathrm{Jy~km~s}^{-1}}\right) \left(\frac{D_\mathrm{L}}{\mathrm{Mpc}}\right)^2 M_\odot$ 

We'll assume a Milky Way CO-to-H2 conversion factor

$X_{\mathrm{CO}} = 2 \times 10^{20}~\mathrm{cm}^{-2} \left(\mathrm{K~km~s}^{-1}\right)^{-1}$

and a CO(2-1) to CO(1-0) flux density ratio of $3.2$. Note that these assumptions are metallicity dependent and have important consequences. For a discussion of this relevant to cool core BCGs, see e.g. McNamara+14. 

### Step 1: In CASA v4.5, I fit a Gaussian to the total CO(2-1) line profile

This can be done in ``casaviewer`` interactively. I fit an elliptical region that covers all $>3\sigma$ CO(2-1) emission found in the ALMA beam-corrected map. I then fit a Gaussian to the line extracted from this region. I won't go into how to do that, as it's pretty straightforward and probably evident in the following screenshots. 

<img src="1.png">
<img src="2.png">
<img src="3.png">

### Step 2: Set some basics

In [16]:
import astropy.units as u
import astropy.constants as const

import numpy as np

Set cosmology...

In [17]:
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

Make some units...

In [18]:
emission_integral_units_mJy = u.mJy * u.km / u.s
emission_integral_units_Jy = u.Jy * u.km / u.s

xco_unit = u.cm**(-2) * (u.K * u.km * u.s**(-1))**(-1)

### Step 3: Set parameters for M87

In [19]:
redshift = 0.004283 # Redshift of M87

##### PUT YOUR GAUSSIAN AREA IN mJY * km/s HERE #####

gaussian_area = 817.015
gaussian_area_err = 118.184

##### PUT YOUR CONVERSION FACTOR & RATIO HERE ######

xco = 2.0e20 * xco_unit
ratio = 3.2  # CO(2-1) to CO(1-0) flux density ratio

######################################################


emission_integral = (gaussian_area * emission_integral_units_mJy).to(emission_integral_units_Jy)
emission_integral_err = (gaussian_area_err * emission_integral_units_mJy).to(emission_integral_units_Jy)


### Step 4: Calculate the gas mass...

In [24]:
def gasmass(redshift,xco,ratio,emission_integral):
    xco_mw = 2.0e20 * xco_unit
    ldist = cosmo.luminosity_distance(redshift).to(u.Mpc)
    
    rawmass = 1.05e4 * (xco / xco_mw) * (1 + redshift)**(-1) * (emission_integral / emission_integral_units_Jy) * (ldist/u.Mpc)**2 * (1.0/ratio)
    
    finalmass = rawmass * u.M_sun
    
    print(ldist)
    
    
    return finalmass

In [25]:
mass = gasmass(redshift=redshift,xco=xco,ratio=ratio,emission_integral=emission_integral)
mass_err = gasmass(redshift=redshift,xco=xco,ratio=ratio,emission_integral=emission_integral_err)

18.403803281981897 Mpc
18.403803281981897 Mpc


The total cold gas mass is: 

In [26]:
mass

<Quantity 904124.8466750141 solMass>

$\pm$

In [27]:
mass_err

<Quantity 130784.73575080001 solMass>

So the total mass is

$(9.0 \pm 1.3) \times 10^5~M_\odot$

## Estimating CO(2-1) / H$\alpha$ flux ratios


## tl;dr

**In Box 1**, where we detect CO(2-1) emission, the CO(2-1) flux is

$F_\mathrm{Box1} = \left(5.5 \pm 0.6\right) \times 10^{-18}$ erg s$^{-1}$ cm$^{-2}$ 

for a CO(2-1)-H$\alpha$ flux ratio of $0.006$. 


**In Box 2**, closer to the core, we don't detect CO(2-1) flux. The $3\sigma$ upper limit on the CO(2-1) flux is

$F_\mathrm{Box2} < 1.3\times10^{-18}$ erg s$^{-1}$ cm$^{-2}$ 

for an upper-limit CO(2-1)-H$\alpha$ flux ratio of $0.001$. 

*Note: These flux ratios assume that Becky's F606N flux is pure H$\alpha$, which isn't the case. We'll correct this once Becky is able to find the spectra*. 


### Details of the calculation

In Aurora's Box 1 (i.e., beyond the edge of the lobe, where we see CO flux), 
```
box(12:30:51.676,+12:23:05.29,9",9",30)```
the emission integral, as measured via CASA by fitting a gaussian to the CO(2-1) line extracted from this box, is

In [9]:
box1_ei = 5.55222e+08 * u.mJy * u.Hz
box1_ei_err = 6.08792e+07 * u.mJy * u.Hz

In [10]:
box1_flux = box1_ei.to(u.erg * u.cm**-2 * u.s**-1, equivalencies=u.spectral)
box1_flux

<Quantity 5.55222e-18 erg / (cm2 s)>

In [11]:
box1_flux_err = box1_ei_err.to(u.erg * u.cm**-2 * u.s**-1, equivalencies=u.spectral)
box1_flux_err

<Quantity 6.08792e-19 erg / (cm2 s)>

In [12]:
becky_F660N_box1_flux = 8.6e-16 * (u.erg * u.cm**-2 * u.s**-1)

In [13]:
box1_flux / becky_F660N_box1_flux

<Quantity 0.00645606976744186>

In Aurora's Box 2 (nearer to M87's core), 
```
box(12:30:51.137,+12:23:09.99,9",9",30)```
we have a total non-detection. The RMS flux is then 
$\sqrt{N} \sigma \Delta v$, where $\sigma$ is the RMS noise per channel, $\Delta V$ is the channel width (in Hz), and $N$ is the number of channels over which $>3\sigma$ emission was detected. The $3\sigma$ upper limit is then just 3 times this number. 

In [14]:
rms = 2.304e-04 * u.Jy

In [15]:
rms_mJy = (rms * 3.0).to(u.mJy)
rms_mJy

<Quantity 0.6912 mJy>

In [140]:
channel_width = ((2.2992e+11 * u.Hz) - (2.29037e+11 * u.Hz))/24
channel_width

<Quantity 36791666.666666664 Hz>

In [141]:
num_channels = 3

In [142]:
threesig_upperlimit_flux = (3.0 * (np.sqrt(num_channels) * (rms_mJy) * channel_width)).to(u.erg * u.cm**-2 * u.s**-1, equivalencies=u.spectral)

In [143]:
threesig_upperlimit_flux


<Quantity 1.3214023457039872e-18 erg / (cm2 s)>

In [144]:
becky_F660N_box2_flux = 1.1e-15 * (u.erg * u.cm**-2 * u.s**-1)

In [145]:
threesig_upperlimit_flux / becky_F660N_box2_flux

<Quantity 0.0012012748597308976>